The Laplacian operator on the surface of a sphere of radius $R$ has the form

$$\nabla^2 u =\frac{1}{R^2\cos(\vartheta)}\left[\frac{\partial}{\partial_\vartheta}\left(\cos(\vartheta)\frac{\partial}{\partial_\vartheta} u\right) + \frac{1}{\cos(\vartheta)}\frac{\partial^2u}{\partial\lambda^2}\right]$$

where $\vartheta$ is latitude and $\lambda$ is longitude. For the purposes of filtering, it's much easier and probably qualitatively similar to just use a local Cartesian tangent plane approximation

$$\nabla^2u = \partial_x^2u + \partial_y^2u = \partial_x(\partial_x u) + \partial_y(\partial_yu).$$

The version on the right is in flux form. We want to make sure that $\int\nabla^2u=0$ (because it implies $\int u = \int\bar{u}$). This can be ensured by discretizing in flux form. Doing that has two steps: find $\partial_xu$ on the left and right cell boundaries, then compute $\partial_x(\partial_xu)$ by just subtracting the flux at the left from the flux at the right and then dividing by the cell size. (Same thing for the $y$ derivatives.)

Suppose we're dealing with cell $i$, which has width $h_i$. (This would be found in MOM6's dxCu variable if you're doing an $x$ derivative of $u$; see also dyCu, dxCv, and dyCv.) The cell to the left has width $h_{i-1}$, etc.
The simplest finite-volume approximation of $\partial_x u$ at the left cell boundary is

$$\partial_x u \approx \frac{2(u_i - u_{i-1})}{h_i + h_{i-1}}.$$

A similar formula applies on the right edge of the cell (replace $i$ by $i+1$ in the above formula). 
Subtracting and dividing yields

$$\partial_x(\partial_x u) \approx \frac{1}{h_i}\left[\frac{2(u_{i+1} - u_{i})}{h_{i+1} + h_{i}} - \frac{2(u_i - u_{i-1})}{h_i + h_{i-1}}\right].$$

(Note that MOM6 also has a variable called $h$. It's not the same thing as my notation above. MOM6's $h$ is a vertical cell size; I'm using the notation $h$ for a horizontal spacing.)

In Neverworld2 the cell spacing in the $y$ direction is uniform, so the formula for $y$ would be

$$\partial_y(\partial_yu)\approx \frac{u_{j-1}-2u_j+u_{j+1}}{h^2}.$$

Also the spacing in the $x$ direction only depends on the $y$ location, which simplifies things too. We don't really need to use the non-equispaced formula anywhere, we just need to make sure that we use the correct $h$ when computing $x$ derivatives.

In [ ]:
import numpy as np
def Laplacian2D(field,landMask,dx,dy):
    """
    Computes a Cartesian Laplacian of field. Assumes dy=constant, dx varies in y direction
    Inputs:
    field is a 2D array (x, y) whose Laplacian is computed
    landMask: 2D array, same size as field: 0 if cell is not on land, 1 if it is on land.
    dx is a 1D array, size size as 2nd dimension of field
    dy is constant
    Output:
    Laplacian of field.
    """
    Nx = np.size(field,0)
    Ny = np.size(field,1) # I suppose these could be inputs
    notLand = 1 - landMask
    # first compute Laplacian in y direction. "Right" is north and "Left" is south for this block
    fluxRight = np.zeros((Nx,Ny))
    fluxRight[:,0:Ny-1] = notLand[:,1:Ny]*(field[:,1:Ny] - field[:,0:Ny-1]) # Set flux to zero if on land
    fluxRight[:,Ny-1] = notLand[:,0]*(field[:,0]-field[:,Ny-1]) # Periodic unless there's land in the way
    fluxLeft = np.zeros((Nx,Ny))
    fluxLeft[:,1:Ny] = notLand[:,0:Ny-1]*(field[:,1:Ny] - field[:,0:Ny-1]) # Set flux to zero if on land
    fluxLeft[:,0] = notLand[:,Ny-1]*(field[:,0]-field[:,Ny-1]) # Periodic unless there's land in the way
    OUT = (1/dy**2)*(fluxRight - fluxLeft)
    # Now compute Laplacian in x direction and add it back in
    fluxRight = 0*fluxRight # re-set to zero just to be safe
    fluxLeft = 0*fluxLeft # re-set to zero just to be safe
    fluxRight[0:Nx-1,:] = notLand[1:Nx,:]*(field[1:Nx,:] - field[0:Nx-1,:]) # Set flux to zero if on land
    fluxRight[Nx-1,:] = notLand[0,:]*(field[0,:]-field[Nx-1,:]) # Periodic unless there's land in the way
    fluxLeft[1:Nx,:] = notLand[0:Nx-1,:]*(field[1:Nx,:] - field[0:Nx-1,:]) # Set flux to zero if on land
    fluxLeft[0,:] = notLand[Nx-1,:]*(field[0,:]-field[Nx-1,:]) # Periodic unless there's land in the way
    OUT = OUT + (1/dx**2)*(fluxRight - fluxLeft)
    return OUT*notLand

In [ ]:
Nx = 11
Ny = 21
dy = 1
dx = np.linspace(1,2,Ny)/2
landMask = 0*np.random.randint(0,2,(Nx,Ny))
data = np.random.randn(Nx,Ny)
L = Laplacian2D(data,landMask,dx,dy)